<a href="https://colab.research.google.com/github/cras-lab/OpenAPI/blob/main/SeoulMetroInfo_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

각자 자신의 API 키를 설정한다.

In [43]:
api_key = "42477864496f6b6133356455666f56"  # 이곳에는 각자 본인의 실제 발급받은 API 키를 입력한다.

필요한 모듈을 임포트 한다.

In [ ]:
필요한 모듈을 임포트 한import requests
import pandas as pd
import json다.

지하철 정보를 읽어오는 함수를 정의한다.

In [ ]:


def get_subway_info(api_key, station_name):
    # URL 인코딩을 위해 station_name을 인코딩
    station_name_encoded = requests.utils.quote(station_name)
    # 아래 Open API 명령어를 통해 값을 읽어온다.
    url = f"http://swopenapi.seoul.go.kr/api/subway/{api_key}/json/realtimeStationArrival/0/5/{station_name_encoded}"

    response = requests.get(url)
    if response.status_code == 200:
        # UTF-8로 인코딩된 응답 데이터를 읽어서 디코딩합니다.
        response_text = response.content.decode('utf-8')
        data = json.loads(response_text)

        # 전체 응답 데이터를 디버그하기 위해 출력합니다.
        print("Response Data:", json.dumps(data, indent=4, ensure_ascii=False))

        if "errorMessage" in data and data["errorMessage"]["status"] != 200:
            print("Error:", data["errorMessage"]["message"])
            return None
        else:
            if 'realtimeArrivalList' in data:
                arrivals = data['realtimeArrivalList']

                # realtimeArrivalList 데이터를 출력하여 확인합니다.
                print("Realtime Arrival List:", json.dumps(arrivals, indent=4, ensure_ascii=False))

                # DataFrame으로 변환
                if isinstance(arrivals, list) and len(arrivals) > 0:
                    df = pd.DataFrame(arrivals)
                    return df
                else:
                    print("No arrivals data to convert to DataFrame.")
                    return None
            else:
                print("Error: 'realtimeArrivalList' not found in response")
                return None
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        return None

def save_to_csv(df, filename):
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"Data saved to {filename}")





원하는 값을 설정한다.

In [ ]:
station_name = "서울"  # 검색할 지하철 역 이름
date = "2024-07-20"  # 검색할 날짜 (YYYY-MM-DD 형식)
time = "15:00"  # 검색할 시간 (HH:MM 형식)

함수를 호출한다. 예시를 위해 함수 내에서 읽어온 JSON 값과 pandas 값을 출력하도록 했다.

In [ ]:
df = get_subway_info(api_key, station_name)

df에 담겨있는 값을 "CSV" 파일로 저장한다.

In [ ]:
if df is not None:
    print(df)
    save_to_csv(df, "subway_info.csv")
else:
    print("No data found")